In [1]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier

In [2]:
df = pd.read_csv('../../data/telecom_churn.csv')

In [3]:
df.head()

,State,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn
0,KS,128,415,No,Yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,No,Yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,No,No,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,Yes,No,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,Yes,No,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3333 entries, 0 to 3332
Data columns (total 20 columns):
State                     3333 non-null object
Account length            3333 non-null int64
Area code                 3333 non-null int64
International plan        3333 non-null object
Voice mail plan           3333 non-null object
Number vmail messages     3333 non-null int64
Total day minutes         3333 non-null float64
Total day calls           3333 non-null int64
Total day charge          3333 non-null float64
Total eve minutes         3333 non-null float64
Total eve calls           3333 non-null int64
Total eve charge          3333 non-null float64
Total night minutes       3333 non-null float64
Total night calls         3333 non-null int64
Total night charge        3333 non-null float64
Total intl minutes        3333 non-null float64
Total intl calls          3333 non-null int64
Total intl charge         3333 non-null float64
Customer service calls    3333 non-null int64


In [5]:
pd.factorize?

In [6]:
df['International plan'] = \
    pd.factorize(df['International plan'])[0]
df['Voice mail plan'] = \
    pd.factorize(df['Voice mail plan'])[0]

In [7]:
df.drop('State', axis=1, inplace=True)

In [8]:
df.head()

,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn
0,128,415,0,0,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,107,415,0,0,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,137,415,0,1,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,84,408,1,1,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,75,415,1,1,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


In [9]:
df['Churn'].value_counts()

False    2850
True      483
Name: Churn, dtype: int64

In [10]:
y = df['Churn'].astype('int')

In [11]:
df.drop('Churn', axis=1, inplace=True)

In [12]:
df.shape

(3333, 18)

In [13]:
#from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split, cross_val_score

In [14]:
X_train, X_test, y_train, y_test = \
    train_test_split(df.values, y, test_size=0.3, random_state=17,
                stratify=y)

In [15]:
X_train.shape, X_test.shape

((2333, 18), (1000, 18))

In [16]:
tree = DecisionTreeClassifier(max_depth=3, random_state=17)

In [17]:
%%time
tree.fit(X_train, y_train)

CPU times: user 7.87 ms, sys: 1.61 ms, total: 9.48 ms
Wall time: 9.47 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=17, splitter='best')

In [18]:
tree_predictions = tree.predict(X_test)

In [19]:
from sklearn.metrics import accuracy_score

In [20]:
accuracy_score(y_test, tree_predictions)

0.88700000000000001

In [21]:
1 - y.sum() / y.shape[0]

0.8550855085508551

In [22]:
np.mean(cross_val_score(tree, X_train, y_train, cv=5))

0.90141713613513341

In [23]:
from sklearn.model_selection import GridSearchCV

In [24]:
GridSearchCV?

In [25]:
tree_params = {'max_depth': np.arange(2, 11),
               'max_features': np.arange(2, 19)
              }

tree_grid = GridSearchCV(tree, tree_params, cv=5, n_jobs=-1,
                        verbose=1)



In [26]:
GridSearchCV?

In [27]:
%%time
tree_grid.fit(X_train, y_train)

Fitting 5 folds for each of 153 candidates, totalling 765 fits


[Parallel(n_jobs=-1)]: Done 304 tasks      | elapsed:    0.6s


CPU times: user 572 ms, sys: 52.1 ms, total: 624 ms
Wall time: 1.58 s


[Parallel(n_jobs=-1)]: Done 765 out of 765 | elapsed:    1.5s finished


GridSearchCV(cv=5, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=17, splitter='best'),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'max_depth': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10]), 'max_features': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=1)

In [28]:
tree_grid.best_params_

{'max_depth': 7, 'max_features': 13}

In [29]:
tree_grid.best_score_

0.94384912130304333

In [30]:
tuned_tree_pred = tree_grid.predict(X_test)

In [31]:
accuracy_score(y_test, tuned_tree_pred)

0.92900000000000005

In [32]:
from sklearn.tree import export_graphviz

In [67]:
export_graphviz?

In [33]:
export_graphviz(tree_grid.best_estimator_, 'tree.dot',
               feature_names=df.columns, filled=True)

In [34]:
!dot -Tpng 'tree.dot' -o 'tree.png'

/bin/sh: dot: command not found


In [35]:
!pip install pydot

<img src='tree.jpeg'>

In [68]:
df.head()

,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls
0,128,415,0,0,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1
1,107,415,0,0,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1
2,137,415,0,1,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0
3,84,408,1,1,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2
4,75,415,1,1,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3


In [69]:
df.columns

Index(['Account length', 'Area code', 'International plan', 'Voice mail plan',
       'Number vmail messages', 'Total day minutes', 'Total day calls',
       'Total day charge', 'Total eve minutes', 'Total eve calls',
       'Total eve charge', 'Total night minutes', 'Total night calls',
       'Total night charge', 'Total intl minutes', 'Total intl calls',
       'Total intl charge', 'Customer service calls'],
      dtype='object')

In [36]:
from sklearn.ensemble import RandomForestClassifier

In [38]:
forest = RandomForestClassifier(random_state=17, n_jobs=-1)

In [39]:
forest.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=-1, oob_score=False, random_state=17,
            verbose=0, warm_start=False)

In [41]:
forest_pred = forest.predict(X_test)
accuracy_score(y_test, forest_pred)

0.92900000000000005